## Eurosat Classification Model
- Paul Ritter
- Connor Hite
- Matt Gish

### Installation of Tensorflow datasets and matplotlib

In [ ]:
!pip install -q tfds-nightly tensorflow matplotlib

### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import keras as keras
from tensorflow.keras import datasets, layers, models 
import tensorflow as tf
from tensorflow import keras

import IPython
!pip install -q -U keras-tuner
import kerastuner as kt


### Loading the dataset and resizing the train test split

In [ ]:
image, label = tfds.as_numpy(tfds.load(
    'eurosat',
    split='train', 
    batch_size=-1, 
    as_supervised=True,
    shuffle_files=True
))

images_train, labels_train = [image[:21600], label[:21600]]

images_test, labels_test = [image[21600:], label[21600:]]


# 
images_train, images_test = images_train / 255.0, images_test / 255.0

print(images_train.shape)
print(labels_train.shape)
print(len(images_test))
print(len(labels_test))



## Plot first 5 Images

In [ ]:
class_names = ['AnnualCrop','Forest', 'HerbaceousVegetation', 'Highway', 
'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake' ]

print(class_names)

plt.figure(figsize=(10,10))
for i in range(15):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(images_train[i], cmap=plt.cm.binary)
  plt.xlabel(class_names[labels_train[i]])
plt.show()

### Define CNN Model

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(8, 3, activation='relu', input_shape=(64, 64, 3))),
model.add(layers.MaxPooling2D()),
model.add(layers.Conv2D(16, 3, activation='relu' )),
model.add(layers.MaxPooling2D()),
model.add(layers.Conv2D(32, 3, activation='relu')),
model.add(layers.MaxPooling2D()),
model.add(layers.Conv2D(64, 3, activation='relu')),
model.add(layers.MaxPooling2D()),
#model.add(layers.MaxPooling2D()),
model.add(layers.GlobalAveragePooling2D()),
#model.add(layers.Flatten()), #GlobalAveragePooling2D----------
model.add(layers.Dense(128, activation='relu')),
model.add(layers.Dropout(rate= 0.2)),
model.add(layers.Dense(10))

In [ ]:
model.summary()

### Model Compilation 

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


### Model Fit and Evaluation

In [ ]:
history = model.fit(images_train,
                    labels_train,
                    epochs=30,
                    validation_data=(images_test, labels_test))


In [ ]:
plt.plot(history.history['accuracy'], label='accuracy') 
plt.plot(history.history['val_accuracy'], 
         label = 'val_accuracy') 
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1]) 
plt.legend(loc='lower right')
test_loss, test_acc = model.evaluate(images_test, labels_test, verbose=2) 
print(test_acc)

# Trying out Keras Tuner


In [ ]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
def build_model(hp):
  model = models.Sequential()
  model.add(layers.Conv2D(64, activation='relu', kernel_size=3, input_shape=(64, 64, 3))),
  model.add(layers.MaxPooling2D()),
  model.add(layers.Conv2D(224, 3, activation='relu' )),
  model.add(layers.MaxPooling2D()),
  model.add(layers.Conv2D(64, 3, activation='relu')),
  model.add(layers.MaxPooling2D()),
  model.add(layers.Conv2D(32, 3, activation='relu')),
  model.add(layers.MaxPooling2D()),
  #model.add(layers.MaxPooling2D()),
  model.add(layers.GlobalAveragePooling2D()),
  #model.add(layers.Flatten()), #GlobalAveragePooling2D----------
  model.add(layers.Dense(192, activation='relu')),
  model.add(layers.Dropout(rate= 0.2)),
  model.add(layers.Dense(10))
  #hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4, 1e-5]) 
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  return model


In [ ]:
tuner= RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=50,
    executions_per_trial=2,
    directory = 'presentation4'
    )

In [ ]:
tuner.search(x=images_train,
             y=labels_train,
             epochs=30,
             validation_data=(images_test, labels_test))

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
tuner.search(images_train, labels_train, epochs = 30, validation_data = (images_test, labels_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 5)[0]

#print(f"""
#The hyperparameter search is complete. The optimal number of units in the first densely-connected
#layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
#is {best_hps.get('learning_rate')}.
#""")
tuner.results_summary()

End of Presentation 

In [ ]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
def build_model(hp):
  model = models.Sequential()
  model.add(layers.Conv2D(hp.Int('units', 32,256,32), activation='relu', kernel_size=3, input_shape=(64, 64, 3))),
  model.add(layers.MaxPooling2D()),
  model.add(layers.Conv2D(16, 3, activation='relu' )),
  model.add(layers.MaxPooling2D()),
  model.add(layers.Conv2D(32, 3, activation='relu')),
  model.add(layers.MaxPooling2D()),
  model.add(layers.Conv2D(64, 3, activation='relu')),
  model.add(layers.MaxPooling2D()),
  #model.add(layers.MaxPooling2D()),
  model.add(layers.GlobalAveragePooling2D()),
  #model.add(layers.Flatten()), #GlobalAveragePooling2D----------
  model.add(layers.Dense(128, activation='relu')),
  model.add(layers.Dropout(rate= 0.2)),
  model.add(layers.Dense(10))
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = hp_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  return model

In [ ]:
tuner= RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=1,
    executions_per_trial=3,
    directory = 'my_dir'
    )

In [ ]:
tuner.search(x=images_train,
             y=labels_train,
             epochs=10,
             validation_data=(images_test, labels_test))

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
tuner.search(images_train, labels_train, epochs = 10, validation_data = (images_test, labels_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

#print(f"""
#The hyperparameter search is complete. The optimal number of units in the first densely-connected
#layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
#is {best_hps.get('learning_rate')}.
#""")
tuner.results_summary()